In [17]:
class stim_adj: 
    def __init__(self, V0, V_data, t_data, data_steps, dt, m, n, h, g_Na, g_K, g_L, E_Na, E_K, E_L, guess_a, guess_c, guess_b):
        '''
        args:
            V0 (float): defined in upload.py to be initial voltage
            V_data (array): defined in upload.py to be voltage time series
            t (array): defined in upload.py as all time steps
            data_steps (float): defined in upload.py to be empiracle time step size
            dt (float): retrieved from parent class specifying desired simulation time stepping
            m, n, h (floats): retrieved from parent class as HH param
            g_Na, g_K, g_L, E_Na, E_K, E_K (floats): retrieved from parent class as HH param
            
            guess_a, guess_c, guess_b (floats): retrieved from parent class as optimization initializers 
        '''
        
        #variables from empiracle data
        self.V0 = V0
        self.V_data = V_data
        self.t_data = t_data
        self.data_steps = data_steps
        
        
        #user specified or default
        self.dt  = dt 
        self.g_Na = g_Na
        self.g_K = g_K
        self.g_L = g_L
        self.E_Na = E_Na
        self.E_K = E_K
        self.E_L = E_L
        self.m = m
        self.n = n
        self.h = h
        self.a_init = guess_a
        self.c_init = guess_c
        self.b_init = guess_b
        
        #retrieved simulation parameter
        self.t_sim = np.arange(0, t_final, dt)
        
        #defining optimization parameters
        self.I_params_init = np.array([guess_a, guess_c, guess_b])
       

        
    # Define the HH model helper equations, these need not automatic imput
    def alpha_m(V):
        '''transition rate constant for m-gates (rapid response Na) shut gates opening as a function of voltage'''
        return 0.1 * (V + 40.0) / (1.0 - np.exp(-(V + 40.0) / 10.0))

    def beta_m(V):
        '''transition rate constant for m-gates (rapid response Na) open gates closing as a function of voltage '''
        return 4.0 * np.exp(-(V + 65.0) / 18.0)

    def alpha_h(V):
        '''transition rate constant for h-gates (slow response Na) shut gates opening as a function of voltage'''
        return 0.07 * np.exp(-(V + 65.0) / 20.0)

    def beta_h(V):
        '''transition rate constant for h-gates (slow response Na) open gates closing as a function of voltage '''
        return 1.0 / (1.0 + np.exp(-(V + 35.0) / 10.0))

    def alpha_n(V):
        '''transition rate constant for n-gates (slow response K) shut gates opening as a function of voltage'''
        return 0.01 * (V + 55.0) / (1.0 - np.exp(-(V + 55.0) / 10.0))

    def beta_n(V):
        '''transition rate constant for n-gates (slow response K) open gates closing as a function of voltage '''
        return 0.125 * np.exp(-(V + 65) / 80.0)

 
    def __forward(self, V, I_params, t):
        '''full hodgkin huxley model for an unknown stim.
        Args: 
            V (float): time dependent voltage
            I_params (tuple): optimization dependent I_params
            t (float): iterated time steps
        Returns: 
            dVdt, dmdt, dhdt, dndt (tuple: floats): rate of change for dynamical HH varianbles
        '''
        I = I_params[0]*np.exp(-(t-I_params[2])**2/(2*I_params[1]**2))
        dVdt = (I - self.g_Na * m**3 * h * (V - self.E_Na) - self.g_K * n**4 * (V - self.E_K) - self.g_L * (V - self.E_L)) / self.C_m
        dmdt = alpha_m(V) * (1 - self.m) - beta_m(V) * self.m
        dhdt = alpha_h(V) * (1 - self.h) - beta_h(V) * self.h
        dndt = alpha_n(V) * (1 - self.n) - beta_n(V) * self.n
        return dVdt, dmdt, dhdt, dndt
    
    # Forward Euler to solve IVP
    def integrate_HH(self, I_params):
        '''forward euler method to solve Hodgkin Huxley
        
        Returns: 
            V_record (array): record of voltages for each time step
        '''
        V_record = np.zeros_like(self.t_sim)
        V = self.V0
        
        m = self.m
        n = self.n
        h = self.h
        
        for i in range(len(self.t_sim)):
            V_record[i] = V
            dVdt, dmdt, dhdt, dndt = __forward(V, self.I_params, self.t_sim[i])
            V += dVdt * self.dt
            m += dmdt * self.dt
            h += dhdt * self.dt
            n += dndt * self.dt
        return V_record
    
    def __cost(self, I_params): 
        '''defines optimizaton problem, objective function sought to minimize'''
        cost = 0
        
        V_record = []
        V = self.V0
        
        m = self.m
        n = self.n
        h = self.h
        
        for i in range(len(self.t_sim)):
        
        # forward euler solver 
            V_record.append(V)
        
            dVdt, dmdt, dhdt, dndt = __forward(V,self.m,self.h, self.n, I_params, self.t_sim[i])
            V += dVdt * self.dt
            m += dmdt * self.dt
            h += dhdt * self.dt
            n += dndt * self.dt

        # compute cost at time t_i
            if self.t_sim[i] in self.data_steps:
                j = np.where(self.data_steps == self.t_sim[i])
                cost += (V_record[i] - self.V_data[j])**2 
                  
        cost = cost/len(self.data_steps)

        return cost   
    

    

In [ ]:
s